In [ ]:
print("testing")

In [4]:
import os
import sys
import pandas as pd

In [5]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
from functions import get_dates, get_accesstoken

In [7]:
from talech_keys import grant_type, client_id_1, client_id_2, client_secret_1, client_secret_2, \
    client_version, token_url, refresh_token_1, refresh_token_2, ID_HQ, ID_IFSC, ID_PSL, ID_SAS, ID_TCD

In [193]:
accesstoken = get_accesstoken(client_id_1, client_secret_1, refresh_token_1, client_version, token_url)

token = {
    'securityToken': accesstoken,
    'X-POS-MerchantId': ID_SAS
}

In [194]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import calendar

def get_months_dates(month, year):
    # This returns list of dates of specified lengths of months.
    lst = []
    for i in range(-14, calendar.monthlen(year, month)): 
        date_ = datetime.date(year, month, 1)
        date = date_ + datetime.timedelta(days=i)
        lst.append(date.strftime("%m/%d/%Y 00:00:00"))
    return lst

month = 7
year = 2018
days = get_months_dates(month, year)

print(days)

['06/17/2018 00:00:00', '06/18/2018 00:00:00', '06/19/2018 00:00:00', '06/20/2018 00:00:00', '06/21/2018 00:00:00', '06/22/2018 00:00:00', '06/23/2018 00:00:00', '06/24/2018 00:00:00', '06/25/2018 00:00:00', '06/26/2018 00:00:00', '06/27/2018 00:00:00', '06/28/2018 00:00:00', '06/29/2018 00:00:00', '06/30/2018 00:00:00', '07/01/2018 00:00:00', '07/02/2018 00:00:00', '07/03/2018 00:00:00', '07/04/2018 00:00:00', '07/05/2018 00:00:00', '07/06/2018 00:00:00', '07/07/2018 00:00:00', '07/08/2018 00:00:00', '07/09/2018 00:00:00', '07/10/2018 00:00:00', '07/11/2018 00:00:00', '07/12/2018 00:00:00', '07/13/2018 00:00:00', '07/14/2018 00:00:00', '07/15/2018 00:00:00', '07/16/2018 00:00:00', '07/17/2018 00:00:00', '07/18/2018 00:00:00', '07/19/2018 00:00:00', '07/20/2018 00:00:00', '07/21/2018 00:00:00', '07/22/2018 00:00:00', '07/23/2018 00:00:00', '07/24/2018 00:00:00', '07/25/2018 00:00:00', '07/26/2018 00:00:00', '07/27/2018 00:00:00', '07/28/2018 00:00:00', '07/29/2018 00:00:00', '07/30/201

In [195]:
import datetime
import requests

def json_get_sales_count(obj):
    """Recursively fetch values from nested JSON."""
    arr = {'bakedgoods': {}, 'breakfast': {}, 'lunch': {}}
    itemvalue = None
    categoryname = None
    itemname = None

    def extract(obj, arr, itemvalue, categoryname, itemname):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, itemvalue, categoryname, itemname)
                elif k == 'item':
                    itemkey = v
                elif k == 'soldQuantity':
                    itemvalue = v
                elif k == 'categoryName':
                    if v == 'LUNCH':
                        categoryname = 'lunch'
                    elif v == 'BREAKFAST':
                        categoryname = 'breakfast'
                    elif v == 'BAKED GOODS':
                        categoryname = 'bakedgoods'
                elif k == 'productName':
                    v = v.replace(' + ', '')
                    v = v.replace('To Go', '')
                    v = v.replace('Sit In', '')
                    v = v.replace('/w Mixed Seeds, Nuts & Honey', '')
                    v = v.replace('/w Cranberries, Hazelnuts & Honey', '')
                    v = v.replace('/w Honey', '')
                    v = v.replace(', ', '')
                    v = v.replace('Not Heated', '')
                    v = v.replace('Heated', '')
                    v = v.replace('Soup & Sandwich', '')
                    v = v.replace('Ham Toastie', 'Ham & Cheese Toastie')
                    itemname = v

                if categoryname and itemvalue and itemname:
                    if itemname in arr[categoryname]:
                        arr[categoryname][itemname] += itemvalue
                    else:
                        arr[categoryname][itemname] = itemvalue
                    itemvalue = None
                    categoryname = None
                    itemname = None

        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, itemvalue, categoryname, itemname)
        return arr

    values = extract(obj, arr, itemvalue, categoryname, itemname)
    return values


def get_summary_report(token, startdate, enddate):
    arr = {
        'searchCriteria': {
            'startDate': startdate,
            'endDate': enddate,
            'includedReports': [102]
        }
    }
    result = requests.post(url='https://mapi-eu.talech.com/reports/receiptssummaryreport',
                           json=arr,
                           headers=token)
    items = result.json()
    salesdata = json_get_sales_count(items)
    return salesdata

In [196]:
dailysales = {}
for i in range(0, len(days)):
    d = pd.to_datetime(days[i])
    end = str(d.replace(hour=23).strftime('%m/%d/%Y %H:%M:%S'))
    salesdata = get_summary_report(token, days[i], end)
    dailysales[days[i]] = salesdata
    
dailysales_df = pd.DataFrame(dailysales)
dailysales_df

,06/17/2018 00:00:00,06/18/2018 00:00:00,06/19/2018 00:00:00,06/20/2018 00:00:00,06/21/2018 00:00:00,06/22/2018 00:00:00,06/23/2018 00:00:00,06/24/2018 00:00:00,06/25/2018 00:00:00,06/26/2018 00:00:00,...,07/22/2018 00:00:00,07/23/2018 00:00:00,07/24/2018 00:00:00,07/25/2018 00:00:00,07/26/2018 00:00:00,07/27/2018 00:00:00,07/28/2018 00:00:00,07/29/2018 00:00:00,07/30/2018 00:00:00,07/31/2018 00:00:00
bakedgoods,"{'Almond Financier': 5, 'Mixed Berry Scone': 1...","{'Cruffin': 4, 'Almond Financier': 6, 'Mixed B...","{'Almond Financier': 4, 'Mixed Berry Scone': 1...","{'Almond Financier': 6, 'Mixed Berry Scone': 1...","{'Almond Financier': 9, 'Mixed Berry Scone': 2...","{'Almond Financier': 6, 'Mixed Berry Scone': 2...","{'Almond Financier': 5, 'Mixed Berry Scone': 1...","{'Almond Financier': 4, 'Mixed Berry Scone': 7...","{'Cruffin': 2, 'Almond Financier': 3, 'Mixed B...","{'Cruffin': 5, 'Almond Financier': 7, 'Mixed B...",...,"{'Almond Financier': 5, 'Mixed Berry Scone': 7...","{'Cruffin': 5, 'Almond Financier': 4, 'Mixed B...","{'Almond Financier': 8, 'Mixed Berry Scone': 2...","{'Cruffin': 3, 'Mixed Berry Scone': 18, 'Pain ...","{'Almond Financier': 7, 'Mixed Berry Scone': 2...","{'Cruffin': 10, 'Almond Financier': 7, 'Mixed ...","{'Almond Financier': 8, 'Mixed Berry Scone': 1...","{'Almond Financier': 8, 'Mixed Berry Scone': 1...","{'Cruffin': 2, 'Almond Financier': 9, 'Mixed B...","{'Cruffin': 5, 'Almond Financier': 6, 'Mixed B..."
breakfast,{'Sausage Roll': 2},"{'Sausage Roll': 3, 'Organic Porridge': 6, 'Be...","{'Sausage Roll': 3, 'Organic Porridge': 3, 'Ha...","{'Sausage Roll': 5, 'Organic Porridge': 2, 'Ha...","{'Sausage Roll': 5, 'Ham Cheese Croissant': 6,...","{'Sausage Roll': 8, 'Ham Cheese Croissant': 2,...","{'Sausage Roll': 3, 'Ham Cheese Croissant': 1,...",{},"{'Sausage Roll': 2, 'Organic Porridge': 4, 'Ha...","{'Sausage Roll': 5, 'Ham Cheese Croissant': 3,...",...,"{'Berry & Granola Parfait': 1, 'Granary Toast'...","{'Organic Porridge': 6, 'Granary Toast': 1, 'S...","{'Sausage Roll': 3, 'Organic Porridge': 4, 'Gr...","{'Sausage Roll': 3, 'Organic Porridge': 7, 'Gr...","{'Sausage Roll': 4, 'Ham Cheese Croissant': 3,...","{'Sausage Roll': 6, 'Ham Cheese Croissant': 4,...","{'Sausage Roll': 5, 'Ham Cheese Croissant': 1,...","{'Berry & Granola Parfait': 2, 'Granary Toast'...","{'Sausage Roll': 4, 'Organic Porridge': 3, 'Be...","{'Sausage Roll': 4, 'Organic Porridge': 6, 'Gr..."
lunch,{'Chicken Caesar': 1},"{'Pulled Ham & Gubeen': 3, 'BBLT Bap': 3, 'So...","{'Pulled Ham': 1, 'Pulled Ham & Gubeen': 2, 'B...","{'Pulled Ham & Gubeen': 3, 'BBLT Bap': 1, 'So...","{'BBLT': 1, 'Soup of the Day': 4, 'Falafel Wra...","{'Pulled Ham & Gubeen': 1, 'Ham & Cheese Toast...","{'Pulled Ham & Gubeen': 4, 'Ham & Cheese Toast...",{},"{'Pulled Ham & Gubeen': 1, 'Soup of the Day': ...","{'Pulled Ham & Gubeen': 3, 'Chicken Caesar': 3...",...,{},"{'Chicken Caesar': 2, 'Ham & Cheese Toastie': 1}","{'Pulled Ham & Gubeen': 5, 'Chicken Caesar': 4...","{'Pulled Ham & Gubeen': 1, 'Ham & Cheese Toast...","{'Pulled Ham & Gubeen': 5, 'Chicken Caesar': 4...","{'Pulled Ham & Gubeen': 3, 'Vegan Falafel Wrap...",{'Chicken Caesar': 1},{},"{'Pulled Ham & Gubeen': 1, 'Vegan Falafel Wrap...","{'Pulled Ham & Gubeen': 5, 'Ham & Cheese Toast..."


In [197]:
# class SalesGrowth:
#     def __init__(self, category, day, percentile):
#         self.category = category
#         self.day = day
#         self.percentile = percentile
        
#     def __iter__(self):
#         yield self.category
#         yield self.day
#         yield self.percentile
        
#     @classmethod
#     def columns(cls):
#         return['category', 'day', 'percentile']

# class AllSalesGrowth:
#     def __init__(self):
#         self.lst = []
        
#     def set_item(self, item):
#         self.lst.append(item)
        
#     def get_item(self, category, day):
#         for i in range(len(self.lst)):
#             if self.lst[i].category == category and self.lst[i].day == day:
#                 return self.lst[i]
#         return False
    
#     def get_percentile(self, category, day):
#         for i in range(len(self.lst)):
#             if self.lst[i].category == category and self.lst[i].day == day:
#                     return self.lst[i].percentile
                
#     @property
#     def items(self):
#         return pd.DataFrame(self.lst, columns=SalesGrowth.columns())
                

In [198]:
# growth_json = pd.read_json('data/percentile_change/SAS/{}_{}.json'.format(month, year))
# growth_json

# growths = AllSalesGrowth()

# for category in growth_json.index:
#     for day in growth_json.columns:    
#         percentile = growth_json.loc[category, day]
#         growth = SalesGrowth(category, day, percentile)
#         growths.set_item(growth)
        
# growths.items

In [199]:
weathers_df = pd.DataFrame(pd.read_json('data/historical_weather/07-01-2018_10-19-2020.json'))
weathers_df = weathers_df.dropna()
weathers_df

,temp,weather_id
2018-07-01,20.9,804.0
2018-07-02,21.0,800.0
2018-07-03,19.6,801.0
2018-07-04,23.1,801.0
2018-07-05,19.1,803.0
2018-07-06,19.8,803.0
2018-07-07,22.1,803.0
2018-07-08,22.8,803.0
2018-07-09,19.4,803.0
2018-07-10,19.5,801.0


In [200]:
class Weather:
    def __init__(self, date, temp, weather):
        self.date = date
        self.temp = temp
        self.weather = weather
        
    def __iter__(self):
        yield self.date
        yield self.temp
        yield self.weather
        
    @classmethod
    def columns(cls):
        return['date', 'temp', 'weather']
    
class AllWeather:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        self.arr[item.date] = item
        
    def get_item(self, date):
        if date in self.arr:
            return self.arr[date]
        else:
            return False
    
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Weather.columns())

In [201]:
weathers = AllWeather()

for index in weathers_df.index:
    date = index.strftime('%m/%d/%Y')
    temp = weathers_df.loc[index, 'temp']
    weather_id = weathers_df.loc[index, 'weather_id']
    weather = Weather(date, temp, weather_id)
    weathers.set_item(weather)
    
weathers.items

,date,temp,weather
0,07/01/2018,20.9,804.0
1,07/02/2018,21.0,800.0
2,07/03/2018,19.6,801.0
3,07/04/2018,23.1,801.0
4,07/05/2018,19.1,803.0
5,07/06/2018,19.8,803.0
6,07/07/2018,22.1,803.0
7,07/08/2018,22.8,803.0
8,07/09/2018,19.4,803.0
9,07/10/2018,19.5,801.0


In [202]:
class Date:
    def __init__(self, date):
        self.date = date
        self.dayofweek = pd.Timestamp(pd.to_datetime(date)).dayofweek
        self.weeknum = pd.to_datetime(date).strftime("%W")
        self.day = pd.to_datetime(date).day
        self.month = pd.to_datetime(date).month
        self.year = pd.to_datetime(date).year
        self.holiday = 0
        
    def __iter__(self):
        yield self.dayofweek
        yield self.weeknum
        yield self.day
        yield self.month
        yield self.year
        yield self.holiday
        
    @classmethod
    def columns(cls):
        return['dayofweek', 'weeknum', 'day', 'month', 'year', 'holiday']
        
class Calendar:
    def __init__(self):
        self.arr = {}
        
    def set_date(self, item):
        self.arr[item.date] = item
        
    def get_date(self, date):
        if date in self.arr:
            return self.arr[date]
        else:
            return False
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Date.columns())

In [203]:
url = 'https://holidays.abstractapi.com/v1/'
params = {
    'api_key': '02a0112703a34f5abb1ab898b4f8125a',
    'country': 'IE',
    'year': str(year)
}

result = requests.get(url=url, params=params)
result_json = result.json()
df_holidays = pd.DataFrame(result_json)
df_holidays

,name,name_local,language,description,country,location,type,date,date_year,date_month,date_day,week_day
0,New Year's Day,,,,IE,Ireland,National,01/01/2018,2018,01,01,Monday
1,St. Patrick's Day,,,,IE,Ireland,National,03/17/2018,2018,03,17,Saturday
2,March Equinox,,,,IE,Ireland,Seasonal,03/20/2018,2018,03,20,Tuesday
3,Good Friday,,,,IE,Ireland,Observance,03/30/2018,2018,03,30,Friday
4,Easter,,,,IE,Ireland,Observance,04/01/2018,2018,04,01,Sunday
5,Easter Monday,,,,IE,Ireland,National,04/02/2018,2018,04,02,Monday
6,May Day,,,,IE,Ireland,National,05/07/2018,2018,05,07,Monday
7,June Bank Holiday,,,,IE,Ireland,National,06/04/2018,2018,06,04,Monday
8,June Solstice,,,,IE,Ireland,Seasonal,06/21/2018,2018,06,21,Thursday
9,August Bank Holiday,,,,IE,Ireland,National,08/06/2018,2018,08,06,Monday


In [204]:
class Product:    
    def __init__(self, id, name, category):
        self.id = id
        self.name = name
        self.category = category
            
    def __iter__(self):
        yield self.id
        yield self.name
        yield self.category
    
    @classmethod
    def columns(cls):
        return ['id', 'name', 'category']
        
    
class Sales:
    def __init__(self, soldnum, date, product=None, weather=None):
        self.product = product
        self.date = date
        self.soldnum = soldnum
        self.prevdaysales = []
        self.weather = weather
#         self.growth = growth
        
    def set_prevdaysales(self, quantity):
        self.prevdaysales.append(quantity)
        
    def __iter__(self):
        yield self.product.name
        yield self.product.id
        yield self.product.category
        yield self.date.date
        yield self.date.dayofweek
        yield self.date.weeknum
        yield self.date.day
        yield self.date.month
        yield self.date.year
        yield self.date.holiday
        for i in range(6):
            try:
                prevsales = self.prevdaysales[i]
            except:
                prevsales = 'null'
            yield prevsales
            
        yield self.weather.temp
        yield self.weather.weather
#         yield self.growth.percentile
        yield self.soldnum
        
    @classmethod
    def columns(cls):
        return['name', 'item_id', 'category', 'timestamp', 'dayofweek', 'weeknum', 'day', 'month', 'year',
               'holiday', 'prevdaysales1', 'prevdaysales2', 'prevdaysales3', 'prevdaysales4',
               'prevdaysales5', 'prevdaysales6', 'temperature', 'weather', 'demand']
    
class AllProduct:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        self.arr[item.name] = item
        
    def get_item(self, name):
        if id in self.arr:
            return self.arr[name]
        else:
            return False
        
    def delete(self, name):
        del self.arr[name]
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Product.columns())
    
class Trainingset:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        key = '{}, {}'.format(item.date.date, item.product.name)
        self.arr[key] = item
        
    def get_item(self, name, date):
        key = '{}, {}'.format(date, name)
        if key in self.arr:
            return self.arr[key]
        else:
            return False
    
    def delete(self, name, date):
        key = '{}, {}'.format(date, name)
        del self.arr[key]
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Sales.columns())

In [205]:
import hashlib
m = hashlib.md5()

trset = Trainingset()
products = AllProduct()
calendar = Calendar()
dt_ = None
holiday = 0

products_json = pd.read_json('data/products/products.json')
products_df = pd.DataFrame(products_json)


def transform_category_to_num(category):
    catnum = 0
    if category == 'bakedgoods':
        catnum = 1
    elif category == 'breakfast':
        catnum = 2
    elif category == 'lunch':
        catnum = 3 
    return catnum

def transform_num_to_category(num):
    cat = None
    if num == 1:
        cat = 'bakedgoods'
    elif num == 2:
        cat = 'breakfast'
    elif num == 3:
        cat = 'lunch' 
    return cat
    

for index in products_df.index:
    id = products_df.loc[index, 'id']
    name = products_df.loc[index, 'name']
    category = products_df.loc[index, 'category']
    product = Product(id, name, category)
    products.set_item(product)

    

for date in dailysales_df.columns:
    # if all categories are True, that means the shop
    # sold none of products, which means the shop were closed
    closed = {'bakedgoods': False, 'breakfast': False, 'lunch': False}
    dt_ = pd.to_datetime(date).strftime("%m/%d/%Y")
    dt = Date(dt_)

    for index in df_holidays.index:
        if df_holidays.loc[index, 'date'] == dt_:
            dt.holiday = 1
    
    calendar.set_date(dt)
    
    for category in dailysales_df.index:
        categorysales = dailysales_df.loc[category, date]
        if categorysales == {}:
            closed[category] = True
            
        for itemname, soldnum in categorysales.items():
            product = products.get_item(itemname)
            weather = weathers.get_item(dt_)
            
            if product == False:
                m.update(str(itemname).encode('utf-8'))
                id = str(int(m.hexdigest(), 16))[0:8]
                catnum = transform_category_to_num(category)
                product = Product(id, itemname, catnum)
                products.set_item(product)
            
            if weather == False:
                weather = Weather(dt_, 'null', 'null')
                weathers.set_item(weather)
            
            dateinfo = calendar.get_date(dt_)
            
            sales = Sales(soldnum, dateinfo, product, weather)
            print(sales.product.name)
            trset.set_item(sales)

    if closed == {'bakedgoods': True, 'breakfast': True, 'lunch': True}:
        # if shops seems closed, do nothing
        continue
    else:
        # if the shop is open but seems the product sales is 0,
        # add the sales 0 into the training data
        for index in products.items.index:
            itemname = products.items.loc[index, 'name']
            item = trset.get_item(itemname, dt_)
            if item is False:
                dateinfo = calendar.get_date(dt_)
                prod = products.items.loc[index]
                print(prod)
                weather = weathers.get_item(dt_)
                
                if weather == False:
                    weather = Weather(dt_, 'null', 'null')
                    weathers.set_item(weather)
                    
#                 growth = growths.get_item(product.category, dateinfo.dayofweek)
                sales = Sales(0, dateinfo, prod, weather)
                trset.set_item(sales)
            else:
                continue

Almond Financier
Mixed Berry Scone
Cheesecake Brownie
Pain au Chocolat
Anzac Biscuit
Snickers Boost Ball
Pain au Raisin
Iced Raisin Swirl
Oatmeal & Raisin Cookie
Cinnabun
Sausage Roll
Chicken Caesar
id          18874909
name         Cruffin
category           1
Name: 12, dtype: object
id                  18508001
name        Wholemeal Muffin
category                   1
Name: 13, dtype: object
id                  24170008
name        Organic Porridge
category                   2
Name: 14, dtype: object
id                         29326359
name        Berry & Granola Parfait
category                          2
Name: 15, dtype: object
id               23661806
name        Granary Toast
category                2
Name: 16, dtype: object
id                     29999649
name        Pulled Ham & Gubeen
category                      3
Name: 17, dtype: object
id           22182897
name        BBLT  Bap
category            3
Name: 18, dtype: object
id                 23182376
name        Soup of 

Name: 28, dtype: object
id                  11138126
name        Christmas Pocket
category                   3
Name: 31, dtype: object
id                         33945594
name        Blueberry Buckwheat Pot
category                          2
Name: 32, dtype: object
Almond Financier
Mixed Berry Scone
Cheesecake Brownie
Oatmeal & Raisin Cookie
Pain au Chocolat
Anzac Biscuit
Wholemeal Muffin
Iced Raisin Swirl
Cinnabun
Snickers Boost Ball
Croissant
Sausage Roll
Ham Cheese Croissant
Berry & Granola Parfait
Pulled Ham & Gubeen
Ham & Cheese Toastie
Chicken Caesar
id                18190792
name        Pain au Raisin
category                 1
Name: 6, dtype: object
id          29237831
name         Cruffin
category           1
Name: 12, dtype: object
id                  15516464
name        Organic Porridge
category                   2
Name: 14, dtype: object
id               11770654
name        Granary Toast
category                2
Name: 16, dtype: object
id           27487511
name      

Name: 27, dtype: object
id              16788277
name        Falafel Wrap
category               3
Name: 28, dtype: object
id                 19022266
name        Lemon Financier
category                  1
Name: 29, dtype: object
id          20230910
name          Reuben
category           3
Name: 30, dtype: object
id                  11138126
name        Christmas Pocket
category                   3
Name: 31, dtype: object
id                         33945594
name        Blueberry Buckwheat Pot
category                          2
Name: 32, dtype: object
Cruffin
Almond Financier
Mixed Berry Scone
Croissant
Pain au Chocolat
Anzac Biscuit
Wholemeal Muffin
Snickers Boost Ball
Oatmeal & Raisin Cookie
Pain au Raisin
Cinnabun
Sausage Roll
Organic Porridge
Berry & Granola Parfait
Granary Toast
Ham Cheese Croissant
Pulled Ham & Gubeen
BBLT  Bap
Vegan Falafel Wrap
Soup of the Day
Ham & Cheese Toastie
Chilli Tuna Wrap
Chicken Caesar
id                    41094398
name        Cheesecake Brownie
c

Name: 16, dtype: object
id           11873111
name        BBLT  Bap
category            3
Name: 18, dtype: object
id                 23182376
name        Soup of the Day
category                  3
Name: 19, dtype: object
id            16155406
name        Pulled Ham
category             3
Name: 20, dtype: object
id                  32650892
name        Chilli Tuna Wrap
category                   3
Name: 22, dtype: object
id                        75240713
name        Chicken & Pesto Pocket
category                         3
Name: 23, dtype: object
id          11098183
name            BBLT
category           3
Name: 27, dtype: object
id              16788277
name        Falafel Wrap
category               3
Name: 28, dtype: object
id                 19022266
name        Lemon Financier
category                  1
Name: 29, dtype: object
id          20230910
name          Reuben
category           3
Name: 30, dtype: object
id                  11138126
name        Christmas Pocket
catego

Name: 30, dtype: object
id                  11138126
name        Christmas Pocket
category                   3
Name: 31, dtype: object
id                         33945594
name        Blueberry Buckwheat Pot
category                          2
Name: 32, dtype: object
Almond Financier
Mixed Berry Scone
Cheesecake Brownie
Croissant
Cruffin
Oatmeal & Raisin Cookie
Pain au Chocolat
Anzac Biscuit
Wholemeal Muffin
Pain au Raisin
Cinnabun
Snickers Boost Ball
Sausage Roll
Organic Porridge
Berry & Granola Parfait
Granary Toast
Ham Cheese Croissant
Pulled Ham & Gubeen
Vegan Falafel Wrap
Ham & Cheese Toastie
Chicken Caesar
id                   10640520
name        Iced Raisin Swirl
category                    1
Name: 7, dtype: object
id           43120576
name        BBLT  Bap
category            3
Name: 18, dtype: object
id                 23182376
name        Soup of the Day
category                  3
Name: 19, dtype: object
id            64754890
name        Pulled Ham
category             3
N

Name: 7, dtype: object
id              36860645
name        Sausage Roll
category               2
Name: 10, dtype: object
id                25131522
name        Chicken Caesar
category                 3
Name: 11, dtype: object
id          56455753
name         Cruffin
category           1
Name: 12, dtype: object
id                  22630826
name        Organic Porridge
category                   2
Name: 14, dtype: object
id                         33039476
name        Berry & Granola Parfait
category                          2
Name: 15, dtype: object
id               15945484
name        Granary Toast
category                2
Name: 16, dtype: object
id                     20291065
name        Pulled Ham & Gubeen
category                      3
Name: 17, dtype: object
id           22182897
name        BBLT  Bap
category            3
Name: 18, dtype: object
id                 23182376
name        Soup of the Day
category                  3
Name: 19, dtype: object
id            64754890


Name: 7, dtype: object
id          31084906
name         Cruffin
category           1
Name: 12, dtype: object
id                  22238357
name        Organic Porridge
category                   2
Name: 14, dtype: object
id               23207365
name        Granary Toast
category                2
Name: 16, dtype: object
id           22182897
name        BBLT  Bap
category            3
Name: 18, dtype: object
id                 23182376
name        Soup of the Day
category                  3
Name: 19, dtype: object
id            64754890
name        Pulled Ham
category             3
Name: 20, dtype: object
id                  22129891
name        Chilli Tuna Wrap
category                   3
Name: 22, dtype: object
id                        75240713
name        Chicken & Pesto Pocket
category                         3
Name: 23, dtype: object
id                    65229292
name        Vegan Falafel Wrap
category                     3
Name: 26, dtype: object
id          11098183
name    

Name: 26, dtype: object
id          11098183
name            BBLT
category           3
Name: 27, dtype: object
id              16788277
name        Falafel Wrap
category               3
Name: 28, dtype: object
id                 19022266
name        Lemon Financier
category                  1
Name: 29, dtype: object
id          20230910
name          Reuben
category           3
Name: 30, dtype: object
id                  11138126
name        Christmas Pocket
category                   3
Name: 31, dtype: object
id                         33945594
name        Blueberry Buckwheat Pot
category                          2
Name: 32, dtype: object
Cruffin
Almond Financier
Mixed Berry Scone
Oatmeal & Raisin Cookie
Pain au Chocolat
Anzac Biscuit
Cheesecake Brownie
Wholemeal Muffin
Pain au Raisin
Cinnabun
Snickers Boost Ball
Croissant
Sausage Roll
Ham Cheese Croissant
Organic Porridge
Granary Toast
Berry & Granola Parfait
Pulled Ham & Gubeen
Vegan Falafel Wrap
Chicken Caesar
Ham & Cheese Toastie


In [206]:
trset.items

,name,item_id,category,timestamp,dayofweek,weeknum,day,month,year,holiday,prevdaysales1,prevdaysales2,prevdaysales3,prevdaysales4,prevdaysales5,prevdaysales6,temperature,weather,demand
0,Almond Financier,22624055,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,5
1,Mixed Berry Scone,29657871,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,13
2,Cheesecake Brownie,32829041,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,2
3,Pain au Chocolat,11250357,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,3
4,Anzac Biscuit,29841461,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,6
5,Snickers Boost Ball,34655061,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,2
6,Pain au Raisin,34558433,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,1
7,Iced Raisin Swirl,27779418,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,1
8,Oatmeal & Raisin Cookie,17315954,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,4
9,Cinnabun,33152995,1,06/17/2018,6,24,17,6,2018,0,null,null,null,null,null,null,null,null,5


In [207]:
os.remove("data/products/products.json")
products.items.to_json(r'data/products/products.json')

In [208]:
pd.set_option("display.max_rows", None)
products.items

,id,name,category
0,84617521,Almond Financier,1
1,83523832,Mixed Berry Scone,1
2,27779551,Cheesecake Brownie,1
3,99602993,Pain au Chocolat,1
4,28216422,Anzac Biscuit,1
5,28468091,Snickers Boost Ball,1
6,29039415,Pain au Raisin,1
7,33344100,Iced Raisin Swirl,1
8,27720930,Oatmeal & Raisin Cookie,1
9,14659762,Cinnabun,1


In [209]:
from datetime import datetime, timedelta

for index in trset.items.index:
    count = 0
    currentitem = trset.get_item(trset.items.loc[index, 'name'], trset.items.loc[index, 'timestamp'])
    currentdt = pd.to_datetime(currentitem.date.date)
    name = currentitem.product.name
    
    while len(currentitem.prevdaysales) < 6:
        if count < -20:
            break
        count -= 1
        
        prevdt = currentdt + timedelta(days=count)
        prevdt = prevdt.strftime("%m/%d/%Y")
        previtem = trset.get_item(name, prevdt)
        if previtem is not False:
            currentitem.set_prevdaysales(previtem.soldnum)
        else:
            currentitem.set_prevdaysales(0)
    count = 0

In [210]:
train = Trainingset()

for key in trset.arr:
    item = trset.arr[key]
    if item.prevdaysales == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]:
        continue
    if item.weather.weather == 'null' or item.weather.temp == 'null':
        continue
    elif item.date.month == month:
        train.set_item(item)
        
train.items

,name,item_id,category,timestamp,dayofweek,weeknum,day,month,year,holiday,prevdaysales1,prevdaysales2,prevdaysales3,prevdaysales4,prevdaysales5,prevdaysales6,temperature,weather,demand
0,Almond Financier,27698822,1,07/01/2018,6,26,1,7,2018,0,8,4,1,4,7,3,20.9,804.0,3
1,Mixed Berry Scone,28996134,1,07/01/2018,6,26,1,7,2018,0,9,14,14,15,12,20,20.9,804.0,10
2,Cheesecake Brownie,87689449,1,07/01/2018,6,26,1,7,2018,0,1,0,1,1,2,2,20.9,804.0,2
3,Croissant,27236508,1,07/01/2018,6,26,1,7,2018,0,0,4,3,3,0,2,20.9,804.0,3
4,Anzac Biscuit,22204376,1,07/01/2018,6,26,1,7,2018,0,7,9,8,6,14,6,20.9,804.0,3
5,Wholemeal Muffin,25377308,1,07/01/2018,6,26,1,7,2018,0,2,4,2,2,2,2,20.9,804.0,2
6,Snickers Boost Ball,28593343,1,07/01/2018,6,26,1,7,2018,0,3,12,8,9,2,4,20.9,804.0,5
7,Oatmeal & Raisin Cookie,90446598,1,07/01/2018,6,26,1,7,2018,0,6,3,6,9,3,4,20.9,804.0,3
8,Pain au Raisin,16127141,1,07/01/2018,6,26,1,7,2018,0,1,3,3,3,3,2,20.9,804.0,2
9,Cinnabun,27509776,1,07/01/2018,6,26,1,7,2018,0,3,6,4,4,3,4,20.9,804.0,3


In [211]:
dataset = train.items.drop('name', axis=1)

In [212]:
for index in dataset.index:
    d = dataset.loc[index, 'timestamp']
    timestamp = pd.to_datetime(d).strftime('%Y-%m-%d')
    dataset.loc[index, 'timestamp'] = timestamp

In [213]:
dataset = dataset.drop(['dayofweek', 'weeknum', 'day', 'month', 'year'], axis=1)

In [214]:
dataset

,item_id,category,timestamp,holiday,prevdaysales1,prevdaysales2,prevdaysales3,prevdaysales4,prevdaysales5,prevdaysales6,temperature,weather,demand
0,27698822,1,2018-07-01,0,8,4,1,4,7,3,20.9,804.0,3
1,28996134,1,2018-07-01,0,9,14,14,15,12,20,20.9,804.0,10
2,87689449,1,2018-07-01,0,1,0,1,1,2,2,20.9,804.0,2
3,27236508,1,2018-07-01,0,0,4,3,3,0,2,20.9,804.0,3
4,22204376,1,2018-07-01,0,7,9,8,6,14,6,20.9,804.0,3
5,25377308,1,2018-07-01,0,2,4,2,2,2,2,20.9,804.0,2
6,28593343,1,2018-07-01,0,3,12,8,9,2,4,20.9,804.0,5
7,90446598,1,2018-07-01,0,6,3,6,9,3,4,20.9,804.0,3
8,16127141,1,2018-07-01,0,1,3,3,3,3,2,20.9,804.0,2
9,27509776,1,2018-07-01,0,3,6,4,4,3,4,20.9,804.0,3


In [215]:
dataset.to_csv(r'data/sales_01-07-2018_31-09-2020/SAS/{}_{}.csv'.format(month, year), index=False)

In [216]:
os.system('say "your program has finished"')

0